# Connect Google collab with Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install Ultralytics

In [ ]:
!pip install ultralytics==8.3.15


# Import My trained Model for players and ball but the detection is good for Players only

In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/football_yolo_finetune_from_last2/weights/best.pt")


# Import zip file (Final project Deep Learning)

In [ ]:
zip_path = "/content/drive/MyDrive/Final_Project_Deep_Learning.zip"


# Extract video for Prediction

In [ ]:
import zipfile
import os


extract_path = "/content/videos"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Successfully", extract_path)


✅ Successfully /content/videos


# Prediction

In [ ]:
results = model.predict(
    source="/content/videos/video.mp4",
    conf=0.5,
    show=True,
    project="runs/detect",
    name="Player_only",
    exist_ok=True,
    save=True
)


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/449) /content/videos/video.mp4: 288x512 19 players, 631.9ms
video 1/1 (frame 2/449) /content/videos/video.mp4: 288x512 17 players, 630.2ms
video 1/1 (frame 3/449) /content/videos/video.mp4: 288x512 17 players, 560.0ms
video 1/1 (frame 4/449) /content/videos/video.mp4: 288x512 17 players, 590.6ms
video 1/1 (frame 5/449) /content/videos/video.mp4: 288x512 18 play

# Show Prediction in My Notebook to make sure that the results are good

In [ ]:
!ffmpeg -i /content/runs/detect/Player_only/video.avi -vcodec libx264 /content/runs/detect/Player_only/video.mp4

from IPython.display import HTML
from base64 import b64encode

video_path = "/content/runs/detect/Player_only/video.mp4"


mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=700 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")

# Import Pretrained Model(Ball)

In [ ]:
from ultralytics import YOLO
model_full = YOLO("/content/drive/MyDrive/yolov8m-football_ball_only.pt")


# Prediction for Model (Full_model)

In [ ]:
from ultralytics import YOLO


results = model_full.predict(
    source="/content/videos/video.mp4",
    save=True,
    project="runs/detect",
    name="model_full",
    exist_ok=True,
    show=True
)


# Show Prediction in my Notebook on video

In [ ]:
!ffmpeg -i /content/runs/detect/model_full/video.avi -vcodec libx264 /content/runs/detect/model_full/video.mp4

from IPython.display import HTML
from base64 import b64encode

video_path = "/content/runs/detect/model_full/video.avi"

with open(video_path, "rb") as f:
    mp4 = f.read()

data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=640 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")


# Implement First Tracking Using botsort

In [ ]:
results = model_full.track(
    source="/content/videos/video.mp4",
    conf=0.4,
    iou=0.5,
    persist=True,
    save=True,
    project="/content/runs",
    name="football_tracking",
    tracker="botsort.yaml"
)

# To change Video from format avi to mp4

In [ ]:
!ffmpeg -i /content/runs/football_tracking/video.avi -vcodec libx264 /content/runs/football_tracking/video.mp4

# Show video in my Notebook

In [ ]:
from IPython.display import Video
Video("/content/runs/football_tracking/video.mp4", embed=True)


# Implement Tracking using Bytetrack

In [ ]:
results = model_full.track(
    source="/content/videos/video.mp4",
    conf=0.3,
    iou=0.5,
    persist=True,
    save=True,
    project="/content/runs",
    name="football_tracking_bytetrack",
    tracker="bytetrack.yaml"
)


# Change video format from avi to mp4

In [ ]:
!ffmpeg -i /content/runs/football_tracking_bytetrack/video.avi -vcodec libx264 /content/runs/football_tracking_bytetrack/video.mp4

# Show video in my Notebook

In [ ]:
from IPython.display import Video
Video("/content/runs/football_tracking_bytetrack/video.mp4", embed=True)


# Tracking player who has ball

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
from IPython.display import Video, display


video_path = "/content/videos/video.mp4"
model_path = "/content/drive/MyDrive/yolov8m-football_ball_only.pt"
output_video_path = "/content/runs/football_tracking_possession.mp4"
output_csv_path = "/content/runs/ball_possession_log.csv"


model = YOLO(model_path)

cap = cv2.VideoCapture(video_path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))


names = {0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}

possession_log = []

results = model.track(
    source=video_path,
    conf=0.4,
    iou=0.5,
    persist=True,
    tracker="bytetrack.yaml",
    stream=True
)

frame_num = 0
for frame_data in results:
    frame_num += 1
    frame = frame_data.orig_img.copy()

    if frame_data.boxes.id is None:
        out.write(frame)
        continue

    boxes = frame_data.boxes.xyxy.cpu().numpy()
    classes = frame_data.boxes.cls.cpu().numpy().astype(int)
    ids = frame_data.boxes.id.cpu().numpy().astype(int)

    balls, players = [], []

    for box, cls, track_id in zip(boxes, classes, ids):
        x1, y1, x2, y2 = map(int, box)
        label = names.get(cls, str(cls))

        if cls == 0:
            balls.append((track_id, (x1, y1, x2, y2)))
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
            cv2.putText(frame, "Ball", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        elif cls in [1, 2]:
            players.append((track_id, (x1, y1, x2, y2)))
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"ID {track_id}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    ball_owner_id = None
    min_dist = None

    if len(balls) > 0 and len(players) > 0:
        ball_id, ball_box = balls[0]
        bx1, by1, bx2, by2 = ball_box
        ball_center = np.array([(bx1 + bx2) / 2, (by1 + by2) / 2])

        min_dist = 1e9
        for player_id, player_box in players:
            px1, py1, px2, py2 = player_box
            player_center = np.array([(px1 + px2) / 2, (py1 + py2) / 2])
            dist = np.linalg.norm(ball_center - player_center)
            if dist < min_dist:
                min_dist = dist
                ball_owner_id = player_id


        if min_dist < 80:
            for player_id, player_box in players:
                if player_id == ball_owner_id:
                    px1, py1, px2, py2 = player_box
                    cv2.rectangle(frame, (px1, py1), (px2, py2), (0, 255, 0), 3)
                    cv2.putText(frame, f" Player {player_id}", (px1, py1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


    possession_log.append({
        "frame": frame_num,
        "ball_owner_id": int(ball_owner_id) if ball_owner_id is not None else None,
        "distance_to_ball": float(min_dist) if min_dist is not None else None
    })

    out.write(frame)


cap.release()
out.release()

df = pd.DataFrame(possession_log)
df.to_csv(output_csv_path, index=False)

print("Sucessfully Video")
print(output_video_path)
print("Successufully Ball Possession(Ball Possession Log):")
print(output_csv_path)

display(Video(output_video_path, embed=True))


# Tracking player who has ball , dividing players into two teams ,showing no of passes and possesions frames to two teams

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from IPython.display import Video, display
from collections import defaultdict

model = YOLO("/content/drive/MyDrive/yolov8m-football_ball_only.pt")
video_path = "/content/videos/video.mp4"
output_path = "/content/runs/football_tracking_teams_corrected.mp4"


cap = cv2.VideoCapture(video_path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))


tracker = "botsort.yaml"


results = model.track(
    source=video_path,
    conf=0.4,
    iou=0.5,
    tracker=tracker,
    persist=True,
    stream=True
)


names = {0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}
color_player = (0, 150, 255)
color_goalkeeper = (255, 255, 0)
color_ball = (0, 255, 255)
color_referee = (200, 200, 200)
color_possession = (0, 255, 0)


last_owner_id = None
possession_counter = defaultdict(int)
passes = []
team_colors = {}
team_possession_counter = defaultdict(int)
team_passes_counter = defaultdict(int)


def get_average_color(frame, box):
    x1, y1, x2, y2 = box
    roi = frame[y1:y2, x1:x2]
    if roi.size == 0:
        return np.array([0,0,0])
    return np.mean(roi.reshape(-1,3), axis=0)

def assign_team(player_id, color):
    if player_id not in team_colors:
        if len(team_colors) == 0:
            team_colors[player_id] = color
        else:
            min_dist = 1e9
            assigned_team = None
            for pid, c in team_colors.items():
                dist = np.linalg.norm(color - c)
                if dist < min_dist:
                    min_dist = dist
                    assigned_team = pid
            if min_dist < 40:
                team_colors[player_id] = team_colors[assigned_team]
            else:
                team_colors[player_id] = color
    return team_colors[player_id]

for frame_data in results:
    frame = frame_data.orig_img.copy()

    if frame_data.boxes.id is None:
        out.write(frame)
        continue

    boxes = frame_data.boxes.xyxy.cpu().numpy()
    classes = frame_data.boxes.cls.cpu().numpy().astype(int)
    ids = frame_data.boxes.id.cpu().numpy().astype(int)

    balls, players = [], []


    for box, cls, track_id in zip(boxes, classes, ids):
        x1, y1, x2, y2 = map(int, box)

        if cls == 0:
            balls.append((track_id, (x1, y1, x2, y2)))
            cv2.rectangle(frame, (x1, y1), (x2, y2), color_ball, 2)
            cv2.putText(frame, "Ball", (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.7, color_ball, 2)

        elif cls in [1,2]:
            avg_color = get_average_color(frame, (x1,y1,x2,y2))
            team_color = assign_team(track_id, avg_color)
            if np.mean(team_color) < 128:
                draw_color = (0,0,255)
                team_name = "Team A"
            else:
                draw_color = (255,0,0)
                team_name = "Team B"
            players.append((track_id, (x1,y1,x2,y2), team_name))
            cv2.rectangle(frame, (x1, y1), (x2, y2), draw_color, 2)
            cv2.putText(frame, f"{team_name} #{track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.7, draw_color, 2)

        else:
            cv2.rectangle(frame, (x1, y1), (x2, y2), color_referee, 2)
            cv2.putText(frame, "Referee", (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.6, color_referee, 2)


    current_owner_id = None
    current_owner_team = None
    if len(balls) > 0 and len(players) > 0:
        bx1, by1, bx2, by2 = balls[0][1]
        ball_center = np.array([(bx1 + bx2)/2, (by1+by2)/2])

        min_dist = 1e9
        for player_id, box, team_name in players:
            px1, py1, px2, py2 = box
            player_center = np.array([(px1 + px2)/2, (py1+py2)/2])
            dist = np.linalg.norm(ball_center - player_center)
            if dist < min_dist:
                min_dist = dist
                current_owner_id = player_id
                current_owner_team = team_name

        if min_dist < 90:
            possession_counter[current_owner_id] += 1
            team_possession_counter[current_owner_team] += 1


            if last_owner_id is not None and current_owner_id != last_owner_id:
                passes.append((last_owner_id, current_owner_id))
                team_passes_counter[current_owner_team] += 1
            last_owner_id = current_owner_id


    if current_owner_id is not None:
        for player_id, box, team_name in players:
            if player_id == current_owner_id:
                px1, py1, px2, py2 = box
                cv2.rectangle(frame, (px1, py1), (px2, py2), color_possession, 4)
                cv2.putText(frame, f" {team_name} #{player_id} HAS THE BALL",
                            (px1, py1 - 15), cv2.FONT_HERSHEY_COMPLEX, 0.8, color_possession, 3)


    start_y = 30
    for idx, (player_id, count) in enumerate(possession_counter.items()):
        cv2.putText(frame, f"Player {player_id} Possession: {count} frames",
                    (10, start_y + idx*25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,200,255), 2)

    offset = start_y + len(possession_counter)*25 + 10
    for team_name, count in team_possession_counter.items():
        cv2.putText(frame, f"{team_name} Possession: {count} frames",
                    (10, offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        offset += 25
    cv2.putText(frame, f"Total Passes: {len(passes)}", (10, offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)

    out.write(frame)

cap.release()
out.release()

print("Vide Successfully")
print(output_path)
display(Video(output_path, embed=True))


print("\n Ball Possession Summary (Frames Count) - Players:")
for player_id, count in possession_counter.items():
    print(f"Player {player_id}: {count} frames")

print("\n Ball Possession Summary - Teams:")
for team_name, count in team_possession_counter.items():
    print(f"{team_name}: {count} frames")

print("\n Total Passes:")
for i, (from_id, to_id) in enumerate(passes, 1):
    print(f"{i}. Player {from_id} -> Player {to_id}")

print("\n Passes per Team:")
for team_name, count in team_passes_counter.items():
    print(f"{team_name}: {count} passes")
